In [ ]:
import xarray as xr
import zarr

from fsspec.implementations.http import HTTPFileSystem

In [ ]:
fs = HTTPFileSystem()
http_map = fs.get_mapper('http://0.0.0.0:9000')

In [ ]:
for key in ['.zattrs', '.zgroup', '.zmetadata', 'keys']:
    print(key, http_map[key], '\n')

In [ ]:
zg = zarr.open_consolidated(http_map, mode='r')

In [ ]:
zg['air']